# Import

In [4]:
import numpy as np

# Define

In [6]:
def guess_square_error(input_answers, input_weights = None):
    rowwise_mean = np.average(input_answers, axis = 0)
    for a in range(input_answers.shape[0]):
        input_answers[a] -= rowwise_mean

    square_ans = input_answers * input_answers
    if input_weights != None:
        for a in range(input_answers.shape[0]):
            square_ans[a] *= input_weights[a]

    return square_ans.sum()

class SingleEntryMatrix():
    def __init__(self, size):
        self.elements = np.zeros(size + size, dtype = bool)
        for i in range(size[0]):
            for j in range(size[1]):
                self.elements[i][j][i][j] = True
    
class VariableArray():
    def __init__(self, size, mean = 0., std = 1., work_components = 1., is_independent_learning = False):
        self.size = size
        self.values = np.random.normal(mean, std, size)
        self.diff = np.zeros(size)
        self.drop = np.ones(size, dtype = bool)
        if work_components == 1:
            self.work = np.ones(size, dtype = bool)
        else:
            self.work = (np.random.random(size) < work_components)
        
        self.values *= self.work
        
        if is_independent_learning:
            self.eta_m = np.zeros(size)
        else:
            self.eta_m = None

    def move(self, eta = None):
        if eta == None:
            self.values -= self.eta_m * self.diff
        else:
            self.values += eta * self.diff
            
    def zerodiff(self):
        self.diff *= 0.
        
    def set_drop(self, rate):
        self.drop = (np.random.random(size) < rate)

class Sigmoid():
    def trans(self, x):
        out_put = np.zeros([len(x)])
        for i in range(len(x)):
            out_put[i] = 1 / (1 + np.exp(-x[i]))
        
        return out_put
    
    def diff(self, x):
        out_put = np.zeros([len(x), len(x)])
        for i in range(len(x)):
            out_put[i][i] = 1 / (1 + np.exp(x[i]) * (1 + np.exp(-x[i])))
            
        return out_put
    
class Identity():
    def trans(self, x):
        return x
    
    def diff(self, x):
        return np.identity(len(x))

class Layer():
    def __init__(self, layers_from, nodes_n, active_function):
        self.lf = layers_from # list :layers connected before
        self.lt = [] # layers connected after
        self.nodes_n = nodes_n
        self.weights = {}
        self.input_bias = VariableArray([self.nodes_n])
        self.nodes_input = np.zeros([self.nodes_n])
        self.nodes_output = np.zeros([self.nodes_n])
        self.nodes_diff = np.zeros([self.nodes_n, self.nodes_n])
        self.backwards = {}
        self.af = active_function
            
    def forward(self):
        self.nodes_output = self.af.trans(self.nodes_input)

    def jacobi(self):
        self.nodes_diff = self.af.diff(self.nodes_input)
    
class DogikoNN():
    def __init__(self):
        self.pp_linear = {
            "multi" : None,
            "trans" : None
        } # pre-processing linear translation
        self.flow = []
        self.layers = {}
    
    def set_training_data(self, input_datas, input_answers, input_weights = None):
        # type input_data : numpy array
        # type input_answer : numpy array
        self.t_datas = input_datas
        self.t_answers = input_answers
        self.t_weights = input_weights
        self.data_size = self.t_datas.shape[1:]
        self.answer_size = self.t_answers.shape[1:]
        if self.t_weights == None:
            self.t_square_error = guess_square_error(self.t_answers)
        else:
            self.t_square_error = guess_square_error(self.t_answers, self.t_weights)
        
        if self.t_square_error == 0:
            self.t_square_error = 1
            print ("all training data has same result, can't normalize error")
    
    def set_validating_data(self, input_datas, input_answers, input_weights = None):
        # type input_data : numpy array
        # type input_answer : numpy array
        if input_datas.shape[1:] != self.data_size:
            print ("Size of datas should be the same as training data, must set_training_data before set_validating_data!")

        if input_answers.shape[1:] != self.answer_size:
            print ("Size of answers should be the same as training data, must set_training_data before set_validating_data!")
            
        self.v_datas = input_datas
        self.v_answers = input_answers
        self.v_weights = input_weights
        if self.v_weights == None:
            self.v_square_error = guess_square_error(self.v_answers)
        else:
            self.v_square_error = guess_square_error(self.v_answers, self.v_weights)
        
        if self.v_square_error == 0:
            self.v_square_error = 1
            print ("all validating data has same result, can't normalize error")
        
    def define_normalized(self, normalized_algorithm):
        self.normal_alg = normalized_algorithm
    
    def normalized(self, input_datas):
        out_put = np.zeros(input_datas.shape)
        for d in range(len(input_datas)):
            out_put[d] = self.pp_linear["multi"] * (self.t_datas[d] - self.pp_linear["trans"])
        
        return out_put
    
    def layers_clear(self):
        self.layers = {}
        
    def insert_layer(self, name, layer, is_flow_add = True):
        self.layers[name] = layer
        if is_flow_add:
            self.flow.append(name)
        
    def build(self):
        if self.normal_alg == "normal":
            temp_trans_datas = self.t_datas.T
            self.single_data_size = len(temp_trans_datas)
            temp_square_mean_array = np.zeros([self.single_data_size])
            temp_sd = np.zeros([self.single_data_size])
            self.pp_linear["trans"] = np.zeros([self.single_data_size])
            for j in range(self.single_data_size):
                feature = temp_trans_datas[j]
                self.pp_linear["trans"][j] = feature.mean()
                temp_sd[j] = feature.std()
            
            temp_sd[temp_sd == 0] = 1
            self.pp_linear["multi"] = 1 / temp_sd
            
        else:
            print ("unacceptable normalized_algorithm")
            return ("unacceptable normalized_algorithm")
        
        self.n_t_datas = self.normalized(self.t_datas)
        self.n_v_datas = self.normalized(self.v_datas)
        if len(self.flow) != len(set(self.flow)):
            print ("each element in .flow should be unique (layer name)")
            return ("each element in .flow should be unique (layer name)")
        
        self.single_entry_matrix = {}
        for l in range(len(self.flow)):
            ln = self.flow[l]
            self.layers[ln].lt = []
            for lnf in self.layers[ln].lf:
                if lnf == "input source":
                    self.layers[ln].weights[lnf] = VariableArray([self.layers[ln].nodes_n, self.single_data_size])
                    self.layers[ln].backwards[lnf] = np.zeros([self.single_data_size])
                elif ln2 not in self.layers:
                    print ("layer " + lnf + " doesn't exist!")
                else:
                    if self.flow.index(lnf) > l:
                        print ("layer " + lnf + " shoud work before layer " + ln)
                        return ("layer " + lnf + " shoud work before layer " + ln)
                    
                    self.layers[lnf].lt.append(ln)
                    self.layers[ln].weights[lnf] = VariableArray([self.layers[ln].nodes_n, self.layers[lnf].nodes_n])
                    self.layers[ln].backwards[lnf] = np.zeros([self.layers[lnf].nodes_n])
                
                single_entry_matrix[self.layers[ln].weights[ln2].size] = SingleEntryMatrix(self.layers[ln].weights[ln2].size)
        
        self.layers[self.flow[-1]].lt.append("output result")
    
    def t_cost_greadient(predict, answer):
        return (predict - answer) / self.t_square_error
        
    def forward_propagation(self, input_array):
        for l in range(len(self.flow)):
            ln = self.flow[l]
            self.layers[ln].nodes_input *= 0.
            for ln2 in self.layers[ln].lf:
                if ln2 == "input source":
                    self.layers[ln].nodes_input += self.layers[ln].weights[ln2].values.dot(input_array)
                else:
                    self.layers[ln].nodes_input += self.layers[ln].weights[ln2].values.dot(self.layers[ln2].nodes_output)
                    
            self.layers[ln].nodes_input += self.layers[ln].input_bias.values
            self.layers[ln].forward()
            self.layers[ln].jacobi()
            
        return self.layers[self.flow[-1]].nodes_output
    
    def prediction(self, input_data):
        out_put = np.zeros(input_data.shape[:1] + self.answer_size)
        for d in range(input_data.shape[0]):
            out_put[d] = self.forward_propagation(input_data[d])
        
        return out_put
    
    def backward_propagation(self, answer, predict, cost_gradient):
        temp_array = np.zeros(self.layers[self.flow[-1]])
        for l in range(len(self.flow) - 1, -1, -1):
            ln = self.flow[l]
            for lnt in self.layers[ln].lt:
                if lnt == "output result":
                    temp_array += cost_gradient(predict, answer)
                else:
                    temp_array += self.layers[lnt].backward[ln]
            
            self.layers[ln].bias.diff += temp_array
            temp_array = temp_array.dot(self.layers[ln].nodes_diff)
            for lnf in self.layers[ln].lf:
                for i in range(self.layers[ln].weight[lnf].size[0]):
                    for j in range(self.layers[ln].weight[lnf].size[1]):
                        if self.layers[ln].weights[lnf].work[i][j] and self.layers[ln].weights[lnf].drop[i][j]:
                            self.layers[ln].weights[lnf].diff[i][j] += temp_array.dot(
                                single_entry_matrix(self.layers[ln].weights[lnf].size).dot(self.layers[lnf].nodes_output)
                            )
                        
                self.layers[ln].backward[lnf] = temp_array.dot(self.layers[ln].weights[lnf])
                
    def var_zerodiff(self):
        for l in range(len(self.flow)):
            ln = self.flow[l]
            self.layers[ln].bias.zerodiff()
            for lnf in self.layers[ln].lf:
                self.layers[ln].weights[lnf].zerodiff()
    
    def var_move(self, eta):
        for l in range(len(self.flow)):
            ln = self.flow[l]
            self.layers[ln].bias.move(eta)
            for lnf in self.layers[ln].lf:
                self.layers[ln].weights[lnf].move(eta)
                
    def single_batch(self, batch_datas, batch_answers, batch_weight = None, eta = None, g_change = None):
        self.var_zerodiff()
        batch_prediction = self.prediction(batch_datas)
        for d in range(len(batch_datas)):
            backward_propagation(batch_prediction[d], batch_answers[d])
            
        self.var_move(eta)
            
    def single_epoch(self, batch_size, batchs_n = None, g_change = None, shuffle = False):
        batch_allow = int(len(self.n_t_datas)/batch_size)
        if batch_allow == 0:
            print("batch size should smaller than testing data amount")
            return "batch size should smaller than testing data amount"
        

In [62]:
try_NN = DogikoNN()

try_data = np.random.rand(*[3, 2]) * 2 - 1
try_answer = np.random.randint(2, size = [3, 2])
try_NN.set_training_data(try_data, try_answer)
try_NN.set_validating_data(try_data, try_answer)

try_NN.define_normalized("normal")

try_NN.insert_layer(
    'A',
    Layer(
        layers_from = ["input source"],
        nodes_n = 2,
        active_function = Sigmoid()
    )
)

try_NN.insert_layer(
    'B',
    Layer(
        layers_from = ['A'],
        nodes_n = 3,
        active_function = Sigmoid()
    )
)

try_NN.insert_layer(
    'C',
    Layer(
        layers_from = ['A'],
        nodes_n = 4,
        active_function = Sigmoid()
    )
)

try_NN.insert_layer(
    'D',
    Layer(
        layers_from = ['B', 'C'],
        nodes_n = 2,
        active_function = Sigmoid()
    )
)